In [1]:
import numpy as np
import random
import math
import copy
import PySimpleGUI as sg

In [14]:
epsilon = 10 ** (-8)

In [15]:
def cmmdc_nr(a, b):
    if a%b == 0:
        return b
    return cmmdc_nr(b, a%b)

In [16]:
def read_input():
    global file_name
    file_read = open(file_name, "r")
    n = int(file_read.readline())
    a = np.array([])
    for i in range(0, n+1):
        value = float(file_read.readline())
        a = np.append(a, value)
    return (a, n)

In [17]:
def derivative(a, n):
    derivative = np.array([])
    for i in range(1, n+1):
        value = i * a[i]
        derivative = np.append(derivative, value)
    return derivative

In [18]:
def double_derivative(a,n):
    deriv = np.array([])
    double_deriv = np.array([])
    
    deriv = derivative(a,n)
    double_deriv = derivative(deriv, n-1)
    return double_deriv

In [19]:
def horner(a, n, x):
    value = a[n]
    for i in range(n-1, -1, -1):
        value = a[i] + value * x
    return value

In [20]:
def metoda_olver(a, n, starting_points):
    global epsilon
    nrmax = -1
    for i in range(1, n+1):
        if abs(a[i]) > nrmax:
            nrmax = abs(a[i])
    r = (abs(a[0]) + nrmax) / a[0]
    radacini = np.array([])
    
    for counter in range(0, starting_points):
        x = random.uniform(-r, r)
        k = 0
        kmax = 10000

        while (1):
            if abs(horner(derivative(a,n), n-1, x)) <= epsilon:
                break

            ck = (horner(a,n,x)**2)*(horner(double_derivative(a,n),n-2,x))/(horner(derivative(a,n),n-1,x)**3)
            delta_x =  horner(a,n,x) / horner(derivative(a,n),n-1,x) + 1/2*ck
            x = x - delta_x
            k = k + 1

            if abs(delta_x) < epsilon or k > kmax or abs(delta_x) > 10**8:
                break
                
        if abs(delta_x) < epsilon:
            already_in = False
            for i in radacini:
                if abs(i - x) < epsilon:
                    already_in = True
                    break
            if already_in == False:
                radacini = np.append(radacini,x)
    return radacini

In [21]:
def cat_rest_imp(a, b):
    global epsilon
    
    a1 = copy.deepcopy(a)
    b1 = copy.deepcopy(b)
    
    cat = np.array([])
    rest = np.array([])
    na = len(a1) - 1
    nb = len(b1) - 1

    while(1):
        grada = -1
        for i in range(0, na+1):
            if abs(a1[i]) > epsilon:
                grada = i
        gradb = -1
        for i in range(0, nb+1):
            if abs(b1[i]) > epsilon:
                gradb = i
                
        coef = a1[grada] / b1[gradb]

        
        for i in range(0, nb+1):
            b1[i] = b1[i] * coef

        cat = np.append(cat, coef)
        for i in range(0, gradb+1):
            a1[grada-i] -= b1[gradb-i]
                    
        
        for i in range(0, nb+1):
            b1[i] /= coef
                            
        grad = -1
        for i in range(0, na+1):
            if abs(a1[i]) > epsilon:
                grad = i
        if grad == -1:
            break
        if grad < nb:
            rest = a1
            break
            
    for i in range(0, len(rest)):
        if abs(rest[i]) < epsilon:
            rest[i] = 0.0
    rest = np.trim_zeros(rest)
    if len(rest) == 0:
        rest = np.array([0.0])
    a = a1
    b = b1
    return (np.flip(cat), rest)

In [22]:
def cmmdc_polinom(a, b):
    global epsilon
    (cat, rest) = cat_rest_imp(a, b)
    if len(rest) == 1 and abs(rest[0]) < epsilon:
        cmmdc = b[0]
        for i in range(1, len(b)):
            cmmdc = cmmdc_nr(cmmdc, b[0])
        for i in range(0, len(b)):
            b[i] = b[i] / cmmdc
        return b
    return cmmdc_polinom(b, rest)

In [23]:
def draw_interface():
    global epsilon, file_name
     
    numar_puncte = 50
    epsilon = 10 ** (-8)

    layout = [[sg.Text("Introduceti epsilon:")], 
              [sg.In(size=(25, 1), enable_events=True, key="EPS")],
              [sg.Button("GATA")],
              [sg.Text("Introduceti numarul de puncte din care sa se porneasca:")], 
              [sg.In(size=(25, 1), enable_events=True, key="PUNCTE")],
              [sg.Button("OK")],
              
              [sg.Button("INPUT1.TXT")],
              [sg.Button("INPUT2.TXT")],
              [sg.Button("INPUT3.TXT")],
              [sg.Button("INPUT4.TXT")],
              
              [sg.Button("Aflare radacini")],
              [sg.Text(size=(100, 1), key="RADACINI")],
              [sg.Button("Aflare coeficienti polinom")],
              [sg.Text(size=(100, 1), key="POL")],
              [sg.Button("Transformare polinom")],
              [sg.Text(size=(100, 1), key="TRANSPOL")],

              [sg.Button("EXIT")]
             ]
    window = sg.Window("Calcul numeric - Laborator 5", layout)

    while True:
        event, values = window.read()

        if event == "EXIT":
            break
        elif event == "GATA":
            epsilon = float(values["EPS"])
        elif event == "OK":
            numar_puncte = int(values["PUNCTE"])
        elif event == "INPUT1.TXT":
            file_name = "input1.txt"
            (a, n) = read_input()
        elif event == "INPUT2.TXT":
            file_name = "input2.txt"
            (a, n) = read_input()
        elif event == "INPUT3.TXT":
            file_name = "input3.txt"
            (a, n) = read_input()
        elif event == "INPUT4.TXT":
            file_name = "input4.txt"
            (a, n) = read_input()
        elif event == "Aflare radacini":
            window["RADACINI"].update(metoda_olver(a,n,numar_puncte))
        elif event == "Aflare coeficienti polinom":
            window["POL"].update(a)
        elif event == "Transformare polinom":
            cmmdc_a = cmmdc_polinom(a, derivative(a,n))
            (cat, rest) = cat_rest_imp(a, cmmdc_a)
            a = cat
            n = len(a) - 1
            window["TRANSPOL"].update(a)
    window.close()
    
draw_interface()

In [31]:
file_name = "input4.txt"
(a, n) = read_input()
print(metoda_olver(a,n,50))
cmmdc_a = cmmdc_polinom(a, derivative(a,n))
(cat, rest) = cat_rest_imp(a, cmmdc_a)
a = cat
n = len(a) - 1
print(metoda_olver(a,n,50))
np.savetxt("output.txt", metoda_olver(a,n,50))

[1.00000001 0.99999998 2.00000004 2.00000007 1.99999998 2.00000003
 2.00000001 1.00000003 0.99999996 0.99999999 2.00000006 1.99999999]
[1. 2.]


In [113]:
file_name = "input2.txt"
(a, n) = read_input()
print(metoda_olver(a,n,50))
np.savetxt("output.txt", metoda_olver(a,n,50))

[ 1.49999997 -0.99999996  0.66666681  0.14285698]


In [112]:
file_name = "input2.txt"
(a, n) = read_input()
horner(a,n,0.6666666)

1.497354492063785e-07

In [14]:
a = np.array([6, 7, 1])
b = np.array([-6,-5,1])
print(cmmdc_polinom(a,b))

[1 1]
